# Counting Boats from Space - Part 1

In [ ]:
%reload_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
%matplotlib inline

## 1. Import requirements

In [ ]:
import os 
import json
import shapely.geometry
import IPython.display
from xcube_sh.cube import open_cube
from xcube_sh.observers import Observers

In [ ]:
from src.GIS_utils import bbox_from_point
from src.config import CubeConfig
from src.preprocess import preprocess, plot_cube_and_background, save_labels

In [ ]:
data_dir = "/home/jovyan/data" # data directory (path)
RADIUS = 500 # AOI radius in meters

## 2. Specify AOI + time window

In [ ]:
##### Restart ports AL, BE, BG, CY, DK, DE, EE, EL, ES + SI, SK, TR, UA, UK + Straits

In [ ]:
with open(os.path.join(data_dir, 'aoi.json'), 'r') as f:
    aoi_file = json.load(f)
    coords = aoi_file['Misc']
    aoi_idx = 5  #### Edit locations
    aoi_name = list(coords.keys())[aoi_idx]
    first_loc = coords[aoi_name][0]
    print(f"first location: {first_loc}")
    lat, lon = coords[aoi_name][0][0], coords[aoi_name][0][1]
    print(lat,lon)
    lat, lon = 8.87, -79.535
    print('{}/{} {}: {}, {}'.format(aoi_idx+1, len(coords), aoi_name, lat, lon))

In [ ]:
#import matplotlib.pyplot as plt
#plt.figure(1, figsize=(10,5))
#plt.hist([k[:2] for k in aoi_file['Ports'] if '(' not in k])
#plt.show()

In [ ]:
#print(len([k for k in aoi_file['Ports'] if '(' not in k]))

In [ ]:
start_date = '2020-01-01'
end_date = '2020-06-01'

In [ ]:
bbox = bbox_from_point(lat=lat, lon=lon, r=RADIUS) # WGS84 coordinates
IPython.display.GeoJSON(shapely.geometry.box(*bbox).__geo_interface__)

## 3. Load the data in a cube

In [ ]:
cube_config = CubeConfig(dataset_name='S2L1C',
                         band_names=['B03', 'B08', 'CLP'], # GREEN + NIR + Clouds
                         tile_size=[2*RADIUS//10, 2*RADIUS//10],
                         geometry=bbox,
                         time_range=[start_date, end_date],
                         time_period='5D', ##### Set to 1 day
                        )

In [ ]:
cube_config

In [ ]:
request_collector = Observers.request_collector()
#request_collector.stats # statistics for SentinelHub request

In [ ]:
cube = open_cube(cube_config, observer=request_collector)
#cube # Verify the content of the cube

In [ ]:
#type(cube)

In [ ]:
#cube

## 4. Preprocess Cube

In [ ]:
cube, background_ndwi = preprocess(cube, max_cloud_proba=0.2, nans_how='any', verbose=1, plot_NDWI=True)

In [ ]:
#cube

In [ ]:
#plot_cube_and_background(-cube, background_ndwi, t=11, cmap="RdYlBu") # display results for each band

## 5. Annotate imagery + save results (B03, B08, CLP, bg_NDWI)

In [ ]:
label = input().split(',') # comma value separated
label = [int(l) for l in label]
assert len(label) == len(cube.time)
save_labels(cube, background_ndwi, label, lat_lon=(lat,lon), data_dir=os.path.join(data_dir, 'chips'), label_filename=os.path.join(data_dir, 'labels.csv'))